# Building Chatbot Which will Answer on Behalf of me

In [36]:
# Create A Chatbot . Install all Required Packages

from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
from pypdf import PdfReader
import gradio as gr
from pathlib import Path

In [37]:
# Read Project Root Path
# Set the root (commonly use __file__ or manually define)
project_root = Path.cwd().parent

#print(project_root)

In [38]:
# Read the Profile PDF
filePath = project_root / "Resourses" / "Profile.pdf"
pdfReader = PdfReader(filePath)
prof_summary = ""
for page in pdfReader.pages:
    text = page.extract_text()
    if text:
        prof_summary += text

#print(prof_summary)

In [39]:
# Read Summary file
summ_filePath = project_root / "Resourses" / "Summary.txt"
with open(summ_filePath, "r", encoding="utf-8") as f:
    summary = f.read()

#print(summary)

In [40]:
# Load Environment Varaible
load_dotenv(override=True)

import os

openai_api_key=os.getenv('API_TOKEN')
deepseek_base_URL = "https://api.deepseek.com"
openai_client = OpenAI(api_key=openai_api_key, base_url=deepseek_base_URL)

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:14]}")
else:
    print("OpenAI API Key not set - please head to the troubleshooting guide in the setup folder")

OpenAI API Key exists and begins sk-ba892fdbe19


In [41]:
# System Prompt
name = "Parag Agrawal"

system_prompt = (
    f"You are acting as {name}, representing {name} on their website. "
    f"Your role is to answer questions specifically about {name}'s career, background, skills, and experience. "
    f"You must faithfully and accurately portray {name} in all interactions. "
    f"You have access to a detailed summary of {name}'s background and their LinkedIn profile, which you should use to inform your answers. "
    f"Maintain a professional, engaging, and approachable tone, as if you are speaking to a potential client or future employer visiting the site. "
    f"If you are unsure of an answer, it is better to honestly acknowledge that than to guess."

    f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{prof_summary}\n\n"
    f"Using this context, please converse naturally and consistently, always staying in character as {name}."
)
#system_prompt

In [42]:
# Create a Chatbot
def liveChat(message, history):
    messages=[{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    
    response = openai_client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        stream=False
    )
    
    return response.choices[0].message.content

In [43]:
gr.ChatInterface(liveChat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
